In [48]:
import os
import io
import struct
from typing import List, Tuple, Optional
from enum import Enum
import pandas as pd

from blowfish import Blowfish

In [49]:
class PackEntryType(Enum):
  Nop = 0
  Folder = 1
  File = 2

In [50]:
def generate_final_blowfish_key(password: str, salt: bytes) -> bytes:
    """
    Reproduces the 'GenerateFinalBlowfishKey' logic in C#.
    """
    # 1) Limit key length to max of 56
    plain_key_length = min(len(password), 56)

    # 2) Convert password to ASCII bytes
    a_key = password.encode("ascii")

    # 3) Create a 56-byte base key buffer
    b_key = bytearray(56)
    
    # Copy salt into b_key
    # Equivalent to `Array.ConstrainedCopy(salt, 0, bKey, 0, salt.Length)`
    b_key[:len(salt)] = salt

    # 4) Generate the final blowfish key by XOR-ing
    #    the ASCII password bytes with the corresponding part of b_key.
    bf_key = bytearray(plain_key_length)
    for x in range(plain_key_length):
        bf_key[x] = a_key[x] ^ b_key[x]

    return bytes(bf_key)

In [51]:
def read_string_with_length(stream, byte_count: int) -> str:
    """
    Read 'byte_count' bytes, then decode using self._encoding,
    stopping at the first 0 (null terminator) if present.
    """
    buffer = stream.read(byte_count)

    # Find null terminator offset (if any)
    terminator_offset = byte_count
    for i in range(byte_count):
        if buffer[i] == 0:
            terminator_offset = i
            break

    return buffer[:terminator_offset].decode('ascii', errors='replace')

In [52]:
data_pk2_path = os.path.join(os.getcwd(), 'data', 'Data.pk2')
media_pk2_path = os.path.join(os.getcwd(), 'data', 'Media.pk2')
print(data_pk2_path)
print(media_pk2_path)

c:\Users\htdun\Desktop\workspace\pk2-extractor\data\Data.pk2
c:\Users\htdun\Desktop\workspace\pk2-extractor\data\Media.pk2


In [53]:
pk2key = '169841'
salt = [0x03, 0xF8, 0xE4, 0x44, 0x88, 0x99, 0x3F, 0x64, 0xFE, 0x35]

In [54]:
key = generate_final_blowfish_key(pk2key, bytes(salt))
key

b'2\xce\xdd|\xbc\xa8'

In [55]:
blowfish = Blowfish()
blowfish.Initialize(key)

In [56]:
_media_file_stream = open(media_pk2_path, 'rb')

In [57]:
signature = _media_file_stream.read(30)
version =  struct.unpack('<i', _media_file_stream.read(4))[0]
encrypted = _media_file_stream.read(1)
encryption_checksum = _media_file_stream.read(16)
payload = _media_file_stream.read(205)


In [58]:
Header = {
    'signature': signature,
    'version': version,
    'encrypted': encrypted,
    'encryption_checksum': encryption_checksum,
    'payload': payload
}

In [59]:
blowfish_checksum_decoded = "Joymax Pak File"

In [60]:
if blowfish and encrypted == b'\x01':
    temp_checksum = blowfish.Encode(blowfish_checksum_decoded.encode('ascii'))
    if temp_checksum is None or temp_checksum[0] != encryption_checksum[0] or temp_checksum[1] != encryption_checksum[1] or temp_checksum[2] != encryption_checksum[2]:
        raise Exception('Failed to open JoymaxPackFile: The password or salt is wrong.')

In [61]:
def read_block_at(position: int):
    _media_file_stream.seek(position, io.SEEK_SET)
    buffer = _media_file_stream.read(128 * 20)
    if blowfish is not None:
      entry_buffer = io.BytesIO(blowfish.Decode(buffer))
    else:
      entry_buffer = io.BytesIO(buffer)

    entries = []
    for _ in range(20):
      entry = {
        "Type": PackEntryType(entry_buffer.read(1)[0]),
        "Name": read_string_with_length(entry_buffer, 89).rstrip('\0'),
        "CreationTime": struct.unpack('<q', entry_buffer.read(8))[0],
        "ModifyTime": struct.unpack('<q', entry_buffer.read(8))[0],
        "DataPosition": struct.unpack('<q', entry_buffer.read(8))[0],
        "DataSize": struct.unpack('<i', entry_buffer.read(4))[0],
        "NextBlock": struct.unpack('<q', entry_buffer.read(8))[0],
        "Payload": entry_buffer.read(2)
      }
      entries.append(entry)
      # print(entry)

    return {
      "Position": position,
      "Entries": entries
    }


def read_blocks_at(position: int):
    result = []

    block = read_block_at(position)
    result.append(block)

    if block["Entries"][19]["NextBlock"] > 0:
        result.extend(read_blocks_at(block["Entries"][19]["NextBlock"]))

    return result

In [62]:
Root = read_blocks_at(256)

In [63]:
blocks_in_memory = {
  "": Root
}

In [64]:
blocks_in_memory

{'': [{'Position': 256,
   'Entries': [{'Type': <PackEntryType.Folder: 1>,
     'Name': '.',
     'CreationTime': 133336205048819447,
     'ModifyTime': 133336205048819447,
     'DataPosition': 256,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
    {'Type': <PackEntryType.Folder: 1>,
     'Name': 'acobject',
     'CreationTime': 133336205048829450,
     'ModifyTime': 133336205048829450,
     'DataPosition': 2816,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
    {'Type': <PackEntryType.Folder: 1>,
     'Name': 'config',
     'CreationTime': 133336205049179438,
     'ModifyTime': 133336205049179438,
     'DataPosition': 35136,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
    {'Type': <PackEntryType.Folder: 1>,
     'Name': 'Effect',
     'CreationTime': 133336205049679437,
     'ModifyTime': 133336205049679437,
     'DataPosition': 41896,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
   

In [65]:
server_dep = os.path.join('server_dep', 'silkroad', 'textdata')
server_dep

'server_dep\\silkroad\\textdata'

In [66]:
data_text = dict()

## TextUISystem.txt

In [67]:
text_ui_system = os.path.join(server_dep, 'TextUISystem.txt')
text_ui_system

'server_dep\\silkroad\\textdata\\TextUISystem.txt'

In [68]:
def get_entry_buffer(file_path: str) -> Optional[dict]:
  parent_folder_path = os.path.dirname(file_path)
  file_name = os.path.basename(file_path)
  if parent_folder_path not in blocks_in_memory:
    paths = parent_folder_path.split(os.path.sep)
    blocks = blocks_in_memory[""]
    current_path = ""

    for sub_folder_name in paths:
        for block in blocks:
            entries = block["Entries"]
            entry = next((e for e in entries if e["Name"] == sub_folder_name and e["Type"] == PackEntryType.Folder), None)
            if entry is None:
                continue
            
            current_path = os.path.join(current_path, entry["Name"])
            if current_path in blocks_in_memory:
                blocks = blocks_in_memory[current_path]
                break
            
            blocks = read_blocks_at(entry["DataPosition"])
            blocks_in_memory[current_path] = blocks
            break
        
  root = blocks_in_memory[parent_folder_path]
  entry = None
  for entries in root:
    entry = next((x for x in entries["Entries"] if x["Name"] == file_name.lower()), None)
    if entry:
      break

  _media_file_stream.seek(entry["DataPosition"], io.SEEK_SET)
  # _media_file_stream.read(entry["DataSize"])
  buffer = io.BytesIO(_media_file_stream.read(entry["DataSize"])) 

  return buffer
   

def get_lines(file_path: str) -> List[Tuple[str, int]]:

  buffer = get_entry_buffer(file_path)

  text = buffer.read().decode('utf-16', errors="replace")
  lines = text.split('\r\n')
  return lines  


In [69]:
text_ui_system_files = get_lines(text_ui_system)
text_ui_system_files

['TextUISystem_100.txt',
 'TextUISystem_1000.txt',
 'TextUISystem_1100.txt',
 'TextUISystem_1200.txt',
 'TextUISystem_1300.txt',
 'TextUISystem_1400.txt',
 'TextUISystem_1500.txt',
 'TextUISystem_1600.txt',
 'TextUISystem_1700.txt',
 'TextUISystem_1800.txt',
 'TextUISystem_1900.txt',
 'TextUISystem_200.txt',
 'TextUISystem_2000.txt',
 'TextUISystem_2100.txt',
 'TextUISystem_2200.txt',
 'TextUISystem_2300.txt',
 'TextUISystem_2400.txt',
 'TextUISystem_2500.txt',
 'TextUISystem_2600.txt',
 'TextUISystem_2700.txt',
 'TextUISystem_2800.txt',
 'TextUISystem_2900.txt',
 'TextUISystem_300.txt',
 'TextUISystem_3000.txt',
 'TextUISystem_3100.txt',
 'TextUISystem_3200.txt',
 'TextUISystem_3300.txt',
 'TextUISystem_3400.txt',
 'TextUISystem_3500.txt',
 'TextUISystem_3600.txt',
 'TextUISystem_3700.txt',
 'TextUISystem_3800.txt',
 'TextUISystem_3900.txt',
 'TextUISystem_400.txt',
 'TextUISystem_4000.txt',
 'TextUISystem_4100.txt',
 'TextUISystem_4200.txt',
 'TextUISystem_4300.txt',
 'TextUISystem_4

In [70]:
text_ui_system_100 = os.path.join(server_dep, 'TextUISystem_100.txt')
text_ui_system_100

'server_dep\\silkroad\\textdata\\TextUISystem_100.txt'

In [71]:
for xxx in text_ui_system_files:
  file_name = os.path.join(server_dep, xxx)
  lines = get_lines(file_name)
  for idx, line in enumerate(lines):
    values = line.split('\t')
    if len(values) < 11:
      continue
    Service = values[0]
    name_str_id = values[2]
    Data = values[10]
    data_text[name_str_id] = {
      "Service": Service,
      "Data": Data
    }


In [72]:
data_text

{'UIC_STT_FONTNAME': {'Service': '1', 'Data': 'Tahoma'},
 'UIIS_STT_MSG_INSTALL_START': {'Service': '1',
  'Data': 'Cài đặt "Con Đường Tơ Lụa"'},
 'UIIS_STT_MSG_INSTALL_FOLDER': {'Service': '1',
  'Data': 'Cài đặt "Con Đường Tơ Lụa" vào thư mục sau'},
 'UIIS_STT_MSG_INSTALL_END': {'Service': '1',
  'Data': 'Con Đường Tơ Lụa đã được cài đặt thành công'},
 'UIIS_STT_INSTALL_FOLDER': {'Service': '1', 'Data': 'Thư mục cài đặt'},
 'UIIS_STT_INSTALL_FILE': {'Service': '1', 'Data': 'File cài đặt'},
 'UIIS_CTL_INSTALL': {'Service': '1', 'Data': 'Cài đặt'},
 'UIIS_CTL_FIND_FOLDER': {'Service': '1', 'Data': 'Tìm kiếm'},
 'UIIS_CTL_START': {'Service': '1', 'Data': 'Bắt đầu'},
 'UIIS_CTL_CANCEL': {'Service': '1', 'Data': 'Hủy bỏ'},
 'UIIS_CTL_CONFIRM': {'Service': '1', 'Data': 'Đồng ý'},
 'UIIS_MSG_ALREADY_INSTALL': {'Service': '1',
  'Data': 'Con Đường Tơ Lụa đã được cài đặt'},
 'UIIS_MSG_UNPACK_ZIP': {'Service': '1',
  'Data': 'Không thể giải nén Con Đường Tơ Lụa. \\nFile nén bị hỏng.'},
 'UIIS_

## TextZoneName.txt

In [73]:
text_zone_name = os.path.join(server_dep, 'TextZoneName.txt')
text_zone_name

'server_dep\\silkroad\\textdata\\TextZoneName.txt'

In [74]:
text_zone_name_files = get_lines(text_zone_name)
text_zone_name_files

['TextZoneName_1000.txt',
 'TextZoneName_1200.txt',
 'TextZoneName_1400.txt',
 'TextZoneName_1600.txt',
 'TextZoneName_1800.txt',
 'TextZoneName_200.txt',
 'TextZoneName_2000.txt',
 'TextZoneName_2200.txt',
 'TextZoneName_2400.txt',
 'TextZoneName_2600.txt',
 'TextZoneName_2800.txt',
 'TextZoneName_3000.txt',
 'TextZoneName_3200.txt',
 'TextZoneName_3400.txt',
 'TextZoneName_3600.txt',
 'TextZoneName_3800.txt',
 'TextZoneName_400.txt',
 'TextZoneName_4000.txt',
 'TextZoneName_4200.txt',
 'TextZoneName_4400.txt',
 'TextZoneName_4600.txt',
 'TextZoneName_600.txt',
 'TextZoneName_800.txt',
 '']

In [75]:
for xxx in text_zone_name_files:
  file_name = os.path.join(server_dep, xxx)
  lines = get_lines(file_name)
  for idx, line in enumerate(lines):
    values = line.split('\t')
    if len(values) < 11:
      continue
    Service = values[0]
    name_str_id = values[2]
    Data = values[10]
    data_text[name_str_id] = {
      "Service": Service,
      "Data": Data
    }


In [76]:
data_text

{'UIC_STT_FONTNAME': {'Service': '1', 'Data': 'Tahoma'},
 'UIIS_STT_MSG_INSTALL_START': {'Service': '1',
  'Data': 'Cài đặt "Con Đường Tơ Lụa"'},
 'UIIS_STT_MSG_INSTALL_FOLDER': {'Service': '1',
  'Data': 'Cài đặt "Con Đường Tơ Lụa" vào thư mục sau'},
 'UIIS_STT_MSG_INSTALL_END': {'Service': '1',
  'Data': 'Con Đường Tơ Lụa đã được cài đặt thành công'},
 'UIIS_STT_INSTALL_FOLDER': {'Service': '1', 'Data': 'Thư mục cài đặt'},
 'UIIS_STT_INSTALL_FILE': {'Service': '1', 'Data': 'File cài đặt'},
 'UIIS_CTL_INSTALL': {'Service': '1', 'Data': 'Cài đặt'},
 'UIIS_CTL_FIND_FOLDER': {'Service': '1', 'Data': 'Tìm kiếm'},
 'UIIS_CTL_START': {'Service': '1', 'Data': 'Bắt đầu'},
 'UIIS_CTL_CANCEL': {'Service': '1', 'Data': 'Hủy bỏ'},
 'UIIS_CTL_CONFIRM': {'Service': '1', 'Data': 'Đồng ý'},
 'UIIS_MSG_ALREADY_INSTALL': {'Service': '1',
  'Data': 'Con Đường Tơ Lụa đã được cài đặt'},
 'UIIS_MSG_UNPACK_ZIP': {'Service': '1',
  'Data': 'Không thể giải nén Con Đường Tơ Lụa. \\nFile nén bị hỏng.'},
 'UIIS_

## TextQuest_OtherString.txt

In [77]:
text_quest_other_string = os.path.join(server_dep, 'TextQuest_OtherString.txt')
text_quest_other_string

'server_dep\\silkroad\\textdata\\TextQuest_OtherString.txt'

In [78]:
text_quest_other_string_files = get_lines(text_quest_other_string)
text_quest_other_string_files

['TextQuest_OtherString_1000.txt',
 'TextQuest_OtherString_1200.txt',
 'TextQuest_OtherString_1400.txt',
 'TextQuest_OtherString_1600.txt',
 'TextQuest_OtherString_1800.txt',
 'TextQuest_OtherString_200.txt',
 'TextQuest_OtherString_2000.txt',
 'TextQuest_OtherString_2200.txt',
 'TextQuest_OtherString_2400.txt',
 'TextQuest_OtherString_2600.txt',
 'TextQuest_OtherString_2800.txt',
 'TextQuest_OtherString_3000.txt',
 'TextQuest_OtherString_3200.txt',
 'TextQuest_OtherString_3400.txt',
 'TextQuest_OtherString_3600.txt',
 'TextQuest_OtherString_3800.txt',
 'TextQuest_OtherString_400.txt',
 'TextQuest_OtherString_4000.txt',
 'TextQuest_OtherString_4200.txt',
 'TextQuest_OtherString_4400.txt',
 'TextQuest_OtherString_4600.txt',
 'TextQuest_OtherString_4800.txt',
 'TextQuest_OtherString_5000.txt',
 'TextQuest_OtherString_600.txt',
 'TextQuest_OtherString_800.txt',
 '']

In [79]:
for xxx in text_quest_other_string_files:
  file_name = os.path.join(server_dep, xxx)
  lines = get_lines(file_name)
  for idx, line in enumerate(lines):
    values = line.split('\t')
    if len(values) < 11:
      continue
    Service = values[0]
    name_str_id = values[2]
    Data = values[10]
    data_text[name_str_id] = {
      "Service": Service,
      "Data": Data
    }


In [80]:
data_text

{'UIC_STT_FONTNAME': {'Service': '1', 'Data': 'Tahoma'},
 'UIIS_STT_MSG_INSTALL_START': {'Service': '1',
  'Data': 'Cài đặt "Con Đường Tơ Lụa"'},
 'UIIS_STT_MSG_INSTALL_FOLDER': {'Service': '1',
  'Data': 'Cài đặt "Con Đường Tơ Lụa" vào thư mục sau'},
 'UIIS_STT_MSG_INSTALL_END': {'Service': '1',
  'Data': 'Con Đường Tơ Lụa đã được cài đặt thành công'},
 'UIIS_STT_INSTALL_FOLDER': {'Service': '1', 'Data': 'Thư mục cài đặt'},
 'UIIS_STT_INSTALL_FILE': {'Service': '1', 'Data': 'File cài đặt'},
 'UIIS_CTL_INSTALL': {'Service': '1', 'Data': 'Cài đặt'},
 'UIIS_CTL_FIND_FOLDER': {'Service': '1', 'Data': 'Tìm kiếm'},
 'UIIS_CTL_START': {'Service': '1', 'Data': 'Bắt đầu'},
 'UIIS_CTL_CANCEL': {'Service': '1', 'Data': 'Hủy bỏ'},
 'UIIS_CTL_CONFIRM': {'Service': '1', 'Data': 'Đồng ý'},
 'UIIS_MSG_ALREADY_INSTALL': {'Service': '1',
  'Data': 'Con Đường Tơ Lụa đã được cài đặt'},
 'UIIS_MSG_UNPACK_ZIP': {'Service': '1',
  'Data': 'Không thể giải nén Con Đường Tơ Lụa. \\nFile nén bị hỏng.'},
 'UIIS_

## TextData_Object.txt

In [81]:
text_data_object = os.path.join(server_dep, 'TextData_Object.txt')
text_data_object

'server_dep\\silkroad\\textdata\\TextData_Object.txt'

In [82]:
text_data_object_files = get_lines(text_data_object)
text_data_object_files

['TextData_Object_1000.txt',
 'TextData_Object_10000.txt',
 'TextData_Object_10200.txt',
 'TextData_Object_10400.txt',
 'TextData_Object_10600.txt',
 'TextData_Object_10800.txt',
 'TextData_Object_11000.txt',
 'TextData_Object_11200.txt',
 'TextData_Object_11400.txt',
 'TextData_Object_11600.txt',
 'TextData_Object_11800.txt',
 'TextData_Object_1200.txt',
 'TextData_Object_12000.txt',
 'TextData_Object_12200.txt',
 'TextData_Object_12400.txt',
 'TextData_Object_12600.txt',
 'TextData_Object_1400.txt',
 'TextData_Object_1600.txt',
 'TextData_Object_1800.txt',
 'TextData_Object_200.txt',
 'TextData_Object_2000.txt',
 'TextData_Object_2200.txt',
 'TextData_Object_2400.txt',
 'TextData_Object_2600.txt',
 'TextData_Object_2800.txt',
 'TextData_Object_3000.txt',
 'TextData_Object_3200.txt',
 'TextData_Object_3400.txt',
 'TextData_Object_3600.txt',
 'TextData_Object_3800.txt',
 'TextData_Object_400.txt',
 'TextData_Object_4000.txt',
 'TextData_Object_4200.txt',
 'TextData_Object_4400.txt',
 '

In [83]:
for xxx in text_data_object_files:
  file_name = os.path.join(server_dep, xxx)
  lines = get_lines(file_name)
  for idx, line in enumerate(lines):
    values = line.split('\t')
    if len(values) < 11:
      continue
    Service = values[0]
    name_str_id = values[2]
    Data = values[10]
    data_text[name_str_id] = {
      "Service": Service,
      "Data": Data
    }
data_text

{'UIC_STT_FONTNAME': {'Service': '1', 'Data': 'Tahoma'},
 'UIIS_STT_MSG_INSTALL_START': {'Service': '1',
  'Data': 'Cài đặt "Con Đường Tơ Lụa"'},
 'UIIS_STT_MSG_INSTALL_FOLDER': {'Service': '1',
  'Data': 'Cài đặt "Con Đường Tơ Lụa" vào thư mục sau'},
 'UIIS_STT_MSG_INSTALL_END': {'Service': '1',
  'Data': 'Con Đường Tơ Lụa đã được cài đặt thành công'},
 'UIIS_STT_INSTALL_FOLDER': {'Service': '1', 'Data': 'Thư mục cài đặt'},
 'UIIS_STT_INSTALL_FILE': {'Service': '1', 'Data': 'File cài đặt'},
 'UIIS_CTL_INSTALL': {'Service': '1', 'Data': 'Cài đặt'},
 'UIIS_CTL_FIND_FOLDER': {'Service': '1', 'Data': 'Tìm kiếm'},
 'UIIS_CTL_START': {'Service': '1', 'Data': 'Bắt đầu'},
 'UIIS_CTL_CANCEL': {'Service': '1', 'Data': 'Hủy bỏ'},
 'UIIS_CTL_CONFIRM': {'Service': '1', 'Data': 'Đồng ý'},
 'UIIS_MSG_ALREADY_INSTALL': {'Service': '1',
  'Data': 'Con Đường Tơ Lụa đã được cài đặt'},
 'UIIS_MSG_UNPACK_ZIP': {'Service': '1',
  'Data': 'Không thể giải nén Con Đường Tơ Lụa. \\nFile nén bị hỏng.'},
 'UIIS_

## TextData_Equip&Skill.txt

In [84]:
text_data_equip_skill = os.path.join(server_dep, 'TextData_Equip&Skill.txt')
text_data_equip_skill

'server_dep\\silkroad\\textdata\\TextData_Equip&Skill.txt'

In [85]:
text_data_equip_skill_files = get_lines(text_data_equip_skill)
text_data_equip_skill_files

['TextData_Equip&Skill_10050.txt',
 'TextData_Equip&Skill_1050.txt',
 'TextData_Equip&Skill_1200.txt',
 'TextData_Equip&Skill_1350.txt',
 'TextData_Equip&Skill_150.txt',
 'TextData_Equip&Skill_1500.txt',
 'TextData_Equip&Skill_1650.txt',
 'TextData_Equip&Skill_1800.txt',
 'TextData_Equip&Skill_1950.txt',
 'TextData_Equip&Skill_2100.txt',
 'TextData_Equip&Skill_2250.txt',
 'TextData_Equip&Skill_2400.txt',
 'TextData_Equip&Skill_2550.txt',
 'TextData_Equip&Skill_2700.txt',
 'TextData_Equip&Skill_2850.txt',
 'TextData_Equip&Skill_300.txt',
 'TextData_Equip&Skill_3000.txt',
 'TextData_Equip&Skill_3150.txt',
 'TextData_Equip&Skill_3300.txt',
 'TextData_Equip&Skill_3450.txt',
 'TextData_Equip&Skill_3600.txt',
 'TextData_Equip&Skill_3750.txt',
 'TextData_Equip&Skill_3900.txt',
 'TextData_Equip&Skill_4050.txt',
 'TextData_Equip&Skill_4200.txt',
 'TextData_Equip&Skill_4350.txt',
 'TextData_Equip&Skill_450.txt',
 'TextData_Equip&Skill_4500.txt',
 'TextData_Equip&Skill_4650.txt',
 'TextData_Equip

In [86]:
for xxx in text_data_equip_skill_files:
  file_name = os.path.join(server_dep, xxx)
  lines = get_lines(file_name)
  for idx, line in enumerate(lines):
    values = line.split('\t')
    if len(values) < 11:
      continue
    Service = values[0]
    name_str_id = values[2]
    Data = values[10]
    data_text[name_str_id] = {
      "Service": Service,
      "Data": Data
    }
data_text

{'UIC_STT_FONTNAME': {'Service': '1', 'Data': 'Tahoma'},
 'UIIS_STT_MSG_INSTALL_START': {'Service': '1',
  'Data': 'Cài đặt "Con Đường Tơ Lụa"'},
 'UIIS_STT_MSG_INSTALL_FOLDER': {'Service': '1',
  'Data': 'Cài đặt "Con Đường Tơ Lụa" vào thư mục sau'},
 'UIIS_STT_MSG_INSTALL_END': {'Service': '1',
  'Data': 'Con Đường Tơ Lụa đã được cài đặt thành công'},
 'UIIS_STT_INSTALL_FOLDER': {'Service': '1', 'Data': 'Thư mục cài đặt'},
 'UIIS_STT_INSTALL_FILE': {'Service': '1', 'Data': 'File cài đặt'},
 'UIIS_CTL_INSTALL': {'Service': '1', 'Data': 'Cài đặt'},
 'UIIS_CTL_FIND_FOLDER': {'Service': '1', 'Data': 'Tìm kiếm'},
 'UIIS_CTL_START': {'Service': '1', 'Data': 'Bắt đầu'},
 'UIIS_CTL_CANCEL': {'Service': '1', 'Data': 'Hủy bỏ'},
 'UIIS_CTL_CONFIRM': {'Service': '1', 'Data': 'Đồng ý'},
 'UIIS_MSG_ALREADY_INSTALL': {'Service': '1',
  'Data': 'Con Đường Tơ Lụa đã được cài đặt'},
 'UIIS_MSG_UNPACK_ZIP': {'Service': '1',
  'Data': 'Không thể giải nén Con Đường Tơ Lụa. \\nFile nén bị hỏng.'},
 'UIIS_

## TextQuest_Speech&Name.txt

In [87]:
text_quest_speech_name = os.path.join(server_dep, 'TextQuest_Speech&Name.txt')
text_quest_speech_name

'server_dep\\silkroad\\textdata\\TextQuest_Speech&Name.txt'

In [88]:
text_quest_speech_name_files = get_lines(text_quest_speech_name)
text_quest_speech_name_files

['TextQuest_Speech&Name_100.txt',
 'TextQuest_Speech&Name_1000.txt',
 'TextQuest_Speech&Name_1100.txt',
 'TextQuest_Speech&Name_1200.txt',
 'TextQuest_Speech&Name_1300.txt',
 'TextQuest_Speech&Name_1400.txt',
 'TextQuest_Speech&Name_1500.txt',
 'TextQuest_Speech&Name_1600.txt',
 'TextQuest_Speech&Name_1700.txt',
 'TextQuest_Speech&Name_1800.txt',
 'TextQuest_Speech&Name_1900.txt',
 'TextQuest_Speech&Name_200.txt',
 'TextQuest_Speech&Name_2000.txt',
 'TextQuest_Speech&Name_2100.txt',
 'TextQuest_Speech&Name_2200.txt',
 'TextQuest_Speech&Name_2300.txt',
 'TextQuest_Speech&Name_2400.txt',
 'TextQuest_Speech&Name_2500.txt',
 'TextQuest_Speech&Name_2600.txt',
 'TextQuest_Speech&Name_2700.txt',
 'TextQuest_Speech&Name_2800.txt',
 'TextQuest_Speech&Name_2900.txt',
 'TextQuest_Speech&Name_300.txt',
 'TextQuest_Speech&Name_3000.txt',
 'TextQuest_Speech&Name_3100.txt',
 'TextQuest_Speech&Name_3200.txt',
 'TextQuest_Speech&Name_3300.txt',
 'TextQuest_Speech&Name_3400.txt',
 'TextQuest_Speech&Name

In [89]:
for xxx in text_quest_speech_name_files:
  file_name = os.path.join(server_dep, xxx)
  lines = get_lines(file_name)
  for idx, line in enumerate(lines):
    values = line.split('\t')
    if len(values) < 11:
      continue
    Service = values[0]
    name_str_id = values[2]
    Data = values[10]
    data_text[name_str_id] = {
      "Service": Service,
      "Data": Data
    }
data_text

{'UIC_STT_FONTNAME': {'Service': '1', 'Data': 'Tahoma'},
 'UIIS_STT_MSG_INSTALL_START': {'Service': '1',
  'Data': 'Cài đặt "Con Đường Tơ Lụa"'},
 'UIIS_STT_MSG_INSTALL_FOLDER': {'Service': '1',
  'Data': 'Cài đặt "Con Đường Tơ Lụa" vào thư mục sau'},
 'UIIS_STT_MSG_INSTALL_END': {'Service': '1',
  'Data': 'Con Đường Tơ Lụa đã được cài đặt thành công'},
 'UIIS_STT_INSTALL_FOLDER': {'Service': '1', 'Data': 'Thư mục cài đặt'},
 'UIIS_STT_INSTALL_FILE': {'Service': '1', 'Data': 'File cài đặt'},
 'UIIS_CTL_INSTALL': {'Service': '1', 'Data': 'Cài đặt'},
 'UIIS_CTL_FIND_FOLDER': {'Service': '1', 'Data': 'Tìm kiếm'},
 'UIIS_CTL_START': {'Service': '1', 'Data': 'Bắt đầu'},
 'UIIS_CTL_CANCEL': {'Service': '1', 'Data': 'Hủy bỏ'},
 'UIIS_CTL_CONFIRM': {'Service': '1', 'Data': 'Đồng ý'},
 'UIIS_MSG_ALREADY_INSTALL': {'Service': '1',
  'Data': 'Con Đường Tơ Lụa đã được cài đặt'},
 'UIIS_MSG_UNPACK_ZIP': {'Service': '1',
  'Data': 'Không thể giải nén Con Đường Tơ Lụa. \\nFile nén bị hỏng.'},
 'UIIS_

## TextQuest_QuestString.txt

In [90]:
text_quest_string = os.path.join(server_dep, 'TextQuest_QuestString.txt')
text_quest_string

'server_dep\\silkroad\\textdata\\TextQuest_QuestString.txt'

In [91]:
text_quest_string = get_lines(text_quest_string)
text_quest_string

['TextQuest_QuestString_100.txt',
 'TextQuest_QuestString_1000.txt',
 'TextQuest_QuestString_10000.txt',
 'TextQuest_QuestString_10050.txt',
 'TextQuest_QuestString_10100.txt',
 'TextQuest_QuestString_10150.txt',
 'TextQuest_QuestString_10200.txt',
 'TextQuest_QuestString_10250.txt',
 'TextQuest_QuestString_10300.txt',
 'TextQuest_QuestString_10350.txt',
 'TextQuest_QuestString_10400.txt',
 'TextQuest_QuestString_10450.txt',
 'TextQuest_QuestString_1050.txt',
 'TextQuest_QuestString_10500.txt',
 'TextQuest_QuestString_10550.txt',
 'TextQuest_QuestString_10600.txt',
 'TextQuest_QuestString_10650.txt',
 'TextQuest_QuestString_10700.txt',
 'TextQuest_QuestString_10750.txt',
 'TextQuest_QuestString_10800.txt',
 'TextQuest_QuestString_10850.txt',
 'TextQuest_QuestString_10900.txt',
 'TextQuest_QuestString_10950.txt',
 'TextQuest_QuestString_1100.txt',
 'TextQuest_QuestString_11000.txt',
 'TextQuest_QuestString_11050.txt',
 'TextQuest_QuestString_11100.txt',
 'TextQuest_QuestString_11150.txt

In [92]:
for xxx in text_quest_string:
  file_name = os.path.join(server_dep, xxx)
  lines = get_lines(file_name)
  for idx, line in enumerate(lines):
    values = line.split('\t')
    if len(values) < 11:
      continue
    Service = values[0]
    name_str_id = values[2]
    Data = values[10]
    data_text[name_str_id] = {
      "Service": Service,
      "Data": Data
    }
data_text

{'UIC_STT_FONTNAME': {'Service': '1', 'Data': 'Tahoma'},
 'UIIS_STT_MSG_INSTALL_START': {'Service': '1',
  'Data': 'Cài đặt "Con Đường Tơ Lụa"'},
 'UIIS_STT_MSG_INSTALL_FOLDER': {'Service': '1',
  'Data': 'Cài đặt "Con Đường Tơ Lụa" vào thư mục sau'},
 'UIIS_STT_MSG_INSTALL_END': {'Service': '1',
  'Data': 'Con Đường Tơ Lụa đã được cài đặt thành công'},
 'UIIS_STT_INSTALL_FOLDER': {'Service': '1', 'Data': 'Thư mục cài đặt'},
 'UIIS_STT_INSTALL_FILE': {'Service': '1', 'Data': 'File cài đặt'},
 'UIIS_CTL_INSTALL': {'Service': '1', 'Data': 'Cài đặt'},
 'UIIS_CTL_FIND_FOLDER': {'Service': '1', 'Data': 'Tìm kiếm'},
 'UIIS_CTL_START': {'Service': '1', 'Data': 'Bắt đầu'},
 'UIIS_CTL_CANCEL': {'Service': '1', 'Data': 'Hủy bỏ'},
 'UIIS_CTL_CONFIRM': {'Service': '1', 'Data': 'Đồng ý'},
 'UIIS_MSG_ALREADY_INSTALL': {'Service': '1',
  'Data': 'Con Đường Tơ Lụa đã được cài đặt'},
 'UIIS_MSG_UNPACK_ZIP': {'Service': '1',
  'Data': 'Không thể giải nén Con Đường Tơ Lụa. \\nFile nén bị hỏng.'},
 'UIIS_

In [93]:
df = pd.DataFrame.from_dict(data_text, orient='index')
df.reset_index(inplace=True)
df.rename(columns={'index': 'Key'}, inplace=True)

In [94]:
df

,Key,Service,Data
0,UIC_STT_FONTNAME,1,Tahoma
1,UIIS_STT_MSG_INSTALL_START,1,"Cài đặt ""Con Đường Tơ Lụa"""
2,UIIS_STT_MSG_INSTALL_FOLDER,1,"Cài đặt ""Con Đường Tơ Lụa"" vào thư mục sau"
3,UIIS_STT_MSG_INSTALL_END,1,Con Đường Tơ Lụa đã được cài đặt thành công
4,UIIS_STT_INSTALL_FOLDER,1,Thư mục cài đặt
...,...,...,...
57853,SN_TALK_QNO_N_KT_082_02,1,Giết 308 Lam Băng Binh và 330 Khôi Băng Binh.
57854,SN_TALK_QNO_N_KT_082_03,1,Chúng quá mạnh mẽ so với bạn?
57855,SN_TALK_QNO_N_KT_082_04,1,Giết 308 Lam Băng Binh và 330 Khôi Băng Binh.
57856,SN_TALK_QNO_N_KT_082_05,1,Bạn đã làm rất tốt. Nhưng bạn không được phép ...


In [95]:
csv_file_path = 'data_text.csv'
df.to_csv(csv_file_path, index=False, encoding='utf-8')